In [2]:
%pip install transformers
%pip install scipy
%pip install pandas
%pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

C:\Users\elean\AppData\Local\Temp\ipykernel_16492\2731424343.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\elean\vs-code-workspace\fyp\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
# import EEC data set

df = pd.read_csv('Equity-Evaluation-Corpus.csv', usecols=["Sentence", "Template","Person","Gender","Race"])
df.head()

eec = df.to_numpy()

In [37]:
# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment-latest"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
male_positive = []
male_neutral = []
male_negative = []
female_positive = []
female_neutral = []
female_negative = []

def Average(list): 
    return sum(list) / len(list) 

for i in range(0, eec[:,0].size):
  sentence = eec[i,0]
  gender = eec[i,3]

#   print(gender)

  encoded_sentence = tokenizer(sentence, return_tensors='pt')
  # print(encoded_sentence)

  # sentiment analysis
  # output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
  output = model(**encoded_sentence)

  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  for j in range(len(scores)):

      l = labels[j]
      s = scores[j]

      if (l == "Positive") and (gender == "male"):
         male_positive.append(s)
      elif (l == "Positive") and (gender == "female"):
         female_positive.append(s)
      elif (l == "Neutral") and (gender == "male"):
         male_neutral.append(s)
      elif (l == "Neutral") and (gender == "female"):
         female_neutral.append(s)
      elif (l == "Negative") and (gender == "male"):
         male_negative.append(s)
      elif (l == "Negative") and (gender == "female"):
         female_negative.append(s)

      # print(l,s)
      # print(male_positive)
      # print(female_positive)
      # print(male_neutral)
      # print(female_neutral)
      # print(male_negative)
      # print(female_negative)

print("Average male positive: " + str(Average(male_positive)))
print("Average female positive: " + str(Average(female_positive)))
print("Average male neutral: " + str(Average(male_neutral)))
print("Average female neutral: " + str(Average(female_neutral)))
print("Average male negative: " + str(Average(male_negative)))
print("Average female negative: " + str(Average(female_negative)))


Average male positive: 0.2332735494417311
Average female positive: 0.23219643041860158
Average male neutral: 0.20905393427259428
Average female neutral: 0.1990991359841751
Average male negative: 0.5576725162255267
Average female negative: 0.5687044333279859


In [12]:
encoded_sentence = tokenizer(sentence, return_tensors='pt')
print(encoded_sentence)

{'input_ids': tensor([[    0,   133,  1068,   817,   127,  3795,   619, 15940,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [13]:
# sentiment analysis
# output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
output = model(**encoded_sentence)

scores = output[0][0].detach().numpy()
scores = softmax(scores)

for i in range(len(scores)):

    l = labels[i]
    s = scores[i]
    print(l,s)

Negative 0.91798365
Neutral 0.07491637
Positive 0.007100006
